### Log File ce5

## Search Conditions: To identify periods of unresponsiveness, use the two log files. Filter for:
- "sbatch" command [Column 5]
-  from user 9204 (the test user) [column 1]
- that have return code 1 [column 4]
- execution time > 15 seconds. [response time]

* We are filtering the ce5 slurm scheduler based on user 9204s failed submission that took longer than 15 seconds.

return code
* returncode 0 = job ran successful without any errors
* returncode 1 =  job failed due to a general error in the script or command executed
* returncode 2 =  job failed due to a misuse of shell built-ins or incorrect usage of commands

retry
* retry 0 - Slurm will not attempt to re-run the job if it fails. The job will be marked as failed after the first unsuccessful attempt.
* retry 1 - Slurm will attempt to re-run the job once if it fails. If the job fails again on the second attempt, it will be marked as failed.
* retry 2 - Slurm will attempt to re-run the job twice if it fails. If the job fails after the third attempt, it will be marked as failed.

Command
* sbatch is a command used in the Slurm Workload Manager to submit a batch script for execution. When you use sbatch, you provide a script that contains the commands and resource requests for your job. 

In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime

In [5]:
ce5 = pd.read_csv(r'C:\Users\billy\Documents\NSS_projects\accre-sweet-potato-casserole\notebooks\Data\slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  #nrows=1000
                 )

ce5.head(5)

,0,1,2,3,4,5
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."


In [6]:
ce5[['test', 'response_time']] = ce5[3].str.split(expand=True) #split column 3 by the strings time and number and created new column for each - to separate out the time portion of the column.
ce5 = ce5.drop(columns = [3, 'test']) # dropped the original column 3 and the newly created column 'test' - no longer needed.
ce5['response_time'] = ce5['response_time'].astype(float) # needed to convert the response_time column to float data type.

In [7]:
ce5[0] = pd.to_datetime(ce5[0], format='mixed') #converted the column containing date and time to a datetime object for future manipulation.
ce5 = ce5.rename(columns={0:'Date and Time'}) #renamed the column to 'Date and Time'

In [8]:
ce5[ce5[4] == 'returncode 1']['response_time'].mean()

13.67645109823773

In [9]:
# def try_parsing_date(text):
#     for fmt in ("%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d %H:%M:%S"):
#         try:
#             return datetime.strptime(text, fmt)
#         except ValueError:
#             pass
#     raise ValueError('no valid date format found')
# ce5['datetime'] = ce5[0].apply(lambda x: try_parsing_date(x))

In [10]:
ce5_unresponsive_9204 = ce5[(ce5[5].str.contains('sbatch')) & (ce5[4].str.contains('returncode 1') & (ce5['response_time'] >= 15) & (ce5[1].str.contains('user 9204')))] # we created a new, filtered dataframe based on the conditions.  
ce5_unresponsive_9204

,Date and Time,1,2,4,5,response_time
49958,2020-10-18 06:53:44.272915,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.038464
49972,2020-10-18 06:54:04.322412,user 9204,retry 1,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.048906
50467,2020-10-18 07:47:25.825172,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.082628
50473,2020-10-18 07:47:45.871008,user 9204,retry 1,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.045221
50582,2020-10-18 07:53:33.972840,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.041486
...,...,...,...,...,...,...
4657074,2021-09-24 13:13:11.616619,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.031287
4661025,2021-09-24 18:14:35.862916,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.041436
4661384,2021-09-24 19:13:14.894282,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",20.051321
4726331,2021-10-02 08:14:16.557499,user 9204,retry 0,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr...",19.083227


In [21]:
ce5_unresponsive_9204.to_csv(r'C:\Users\billy\Documents\NSS_projects\accre-sweet-potato-casserole\notebooks\Data\ce5_unresponsive_9204.csv')